In [1]:
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from tensorflow.keras.applications.resnet50 import decode_predictions
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import json
import cv2
import random
import sys
import os
import torch

# Get the absolute path of the src folder
sys.path.append(os.path.abspath("src"))

from utils_attack import compute_psnr, compute_ssim, compute_msssim, eval_psnr_ssim_msssim, bytearray_to_bits
from regen_pipe import ReSDPipeline
from regen_attack import DiffWMAttacker
from utils import show_image, preprocess_image, clip_eps, get_label
from watermark import generate_adversaries_targeted, perturb_image, verify_watermark, collect_logits

2025-03-02 16:42:39.417240: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-02 16:42:39.417262: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-02 16:42:39.418418: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-02 16:42:39.426880: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
wm_text = 'test'
device = 'cuda:0'

print_width = 50

In [3]:
pipe = ReSDPipeline.from_pretrained("stabilityai/stable-diffusion-2-1", torch_dtype=torch.float16, revision="fp16")
pipe.set_progress_bar_config(disable=True)
pipe.to(device)
print('Finished loading model')

model_index.json:   0%|          | 0.00/517 [00:00<?, ?B/s]

/home/amt007/.local/lib/python3.11/site-packages/diffusers/pipelines/pipeline_loading_utils.py:242: FutureWarning: You are loading the variant fp16 from stabilityai/stable-diffusion-2-1 via `revision='fp16'` even though you can load it via `variant=`fp16`. Loading model variants via `revision='fp16'` is deprecated and will be removed in diffusers v1. Please use `variant='fp16'` instead.
  warnings.warn(


Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/681M [00:00<?, ?B/s]

scheduler%2Fscheduler_config.json:   0%|          | 0.00/351 [00:00<?, ?B/s]

...

vae%2Fconfig.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

diffusion_pytorch_model.bin:   0%|          | 0.00/1.73G [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

An error occurred while trying to fetch /tmp/xdg-cache/huggingface/hub/models--stabilityai--stable-diffusion-2-1/snapshots/f7f33030acc57428be85fbec092c37a78231d75a/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /tmp/xdg-cache/huggingface/hub/models--stabilityai--stable-diffusion-2-1/snapshots/f7f33030acc57428be85fbec092c37a78231d75a/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch /tmp/xdg-cache/huggingface/hub/models--stabilityai--stable-diffusion-2-1/snapshots/f7f33030acc57428be85fbec092c37a78231d75a/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /tmp/xdg-cache/huggingface/hub/models--stabilityai--stable-diffusion-2-1/snapshots/f7f33030acc57428be85fbec092c37a78231d75a/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
/opt/conda/lib/python3.11/site-packages/transformers/tokenization_utils_ba

Finished loading model


In [4]:
attackers = {
    'diff_attacker_60': DiffWMAttacker(pipe, batch_size=5, noise_step=60, captions={})
}

Diffuse attack initialized with noise step 60 and use prompt 0


In [8]:
from PIL import Image
# Define paths for the input and output of the attack
# watermarked_img_path = "blackbox_watermarked_image.jpg"  # Input image
watermarked_img_path = "output/perturbed_image.jpg"
attacked_img_path = "perturbed_panda_attacked.jpg"


image = Image.open(watermarked_img_path).convert("RGB")

attacker_name = "diff_attacker_60"
attacker = attackers[attacker_name]   # Select the attacker

print('*' * print_width)
print(f'Attacking PGD WhiteBox Watermark with {attacker_name}')


# Move attacker model to CUDA if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
attacker.pipe.to(device)  # Ensure the model is on the right device

# Apply the attack
attacker.attack([watermarked_img_path], [attacked_img_path])

print('Finished attacking')

**************************************************
Attacking PGD WhiteBox Watermark with diff_attacker_60


0it [00:00, ?it/s]


RuntimeError: Given groups=1, weight of size [128, 3, 3, 3], expected input[1, 4, 224, 224] to have 3 channels, but got 4 channels instead